
# # 02 — Data Processing & Feature Engineering (Early Default Modeling Dataset)
#
# **Objective**
# Build a clean, modeling-ready dataset to predict **early default** for unsecured consumer loans.
#
# **Key outputs**
# - Target variable: `early_default` (Charged Off within 6 months)
# - Modeling population: loans observable for at least 6 months OR early-defaulted within 6 months
# - Feature set: origination-time variables only (to avoid leakage)
# - Encoded dataset saved to: `data/processed/early_default_modeling_dataset.csv`
#
# **Why this notebook exists**
# This notebook freezes the dataset and target definition. Modeling (Logistic/XGBoost, thresholds, metrics)
# is done in a separate notebook (03) for reproducibility and auditability.


In [1]:
import pandas as pd 
import numpy as np 

# ## 1) Load Raw Data
# We load the LendingClub loan-level dataset (Kaggle version). Note: `id` and `member_id` can be fully null
# in some Kaggle mirrors due to anonymization. For this capstone, we model at the **loan level**, so unique IDs
# are not required.

df = pd.read_csv (
    "../data/raw/lendingclub/loan.csv",
    low_memory=False
)

# 2) Quick Column Scan (EDA Support)
# This scan identifies columns related to delinquency, payment behavior, and time-since variables.
# We use this mainly to confirm what exists in the raw dataset and to flag potential leakage variables.
# (This is informational only; we do not use post-origination variables as predictors in the final model.)


In [2]:
[col for col in df.columns if 
    'delinq' in col.lower() or 
    'dpd' in col.lower() or 
    'late' in col.lower() or 
    'pymnt' in col.lower() or 
    'mths' in col.lower() or 
    'num_tl' in col.lower()
]

['pymnt_plan',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_late_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'acc_now_delinq',
 'mths_since_rcnt_il',
 'acc_open_past_24mths',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'sec_app_inq_last_6mths',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med',
 'sec_app_mths_since_last_major_derog',
 'hardship_dpd']

## 3) Inspect Loan Status Distribution
# `loan_status` is used only to define the target label. It must **not** be used as a predictor
# (it contains outcome information).


In [3]:
df['loan_status'].value_counts()


loan_status
Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: count, dtype: int64

# 4) Date Range of Origination
# We use `issue_d` to:
# - build a realistic **temporal split** later (train on past, test on future)
# - align observation windows


In [4]:
df['issue_d'].min(), df['issue_d'].max()


('Apr-2008', 'Sep-2018')

# 5) Parse Date Columns
# LendingClub dates typically appear in formats like "Dec-2015". We specify the parsing format explicitly to:
# - ensure consistency
# - avoid warnings and slow inference

In [5]:
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y', errors='coerce')
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], format='%b-%Y', errors='coerce')


# 6) Approximate Loan "Age" in Months
# We approximate the time from origination to last recorded payment (`last_pymnt_d`).
# This is used ONLY for target construction and observation window filtering.
#
# Note: Months are computed using year/month arithmetic (not timedeltas), because month-length varies.


In [6]:
df['months_to_last_payment'] = (
    (df['last_pymnt_d'].dt.year - df['issue_d'].dt.year) * 12 +
    (df['last_pymnt_d'].dt.month - df['issue_d'].dt.month)
)


# 7) Define Target: Early Default
# **Target definition (capstone):**
# `early_default = 1` if the loan is **Charged Off within 6 months** of origination.
#
# Rationale:
# - Focuses on "early warning" rather than lifetime default.
# - Uses an observable proxy (charge-off timing) available in the dataset.
# - Avoids using post-origination payment variables as predictors (leakage control).
#
# Note: We use "Charged Off" as the default event because it is the strongest economic loss indicator.
# "Default" exists but is extremely rare in this dataset.


In [7]:
df['early_default'] = np.where(
    (
        df['loan_status'].isin(['Charged Off'])
    ) &
    (
        df['months_to_last_payment'] <= 6
    ),
    1,
    0
)


# 8) Define the Modeling Population (Observation Window Control)
# We keep loans that meet either condition:
# 1) They **early-defaulted** within 6 months (we must retain them)
# OR
# 2) They have at least **6 months of observable history** (`months_to_last_payment >= 6`)
#
# This prevents "right-censoring" bias (loans too new to observe 6 months of performance).


In [8]:
df_model = df[
    (df['early_default'] == 1) |
    (df['months_to_last_payment'] >= 6)
].copy()



# 9) (Optional sanity checks)
# We validate:
# - prevalence of the target
# - dataset size after filtering
# These checks help confirm class imbalance and that filtering did not accidentally remove positives.


In [9]:
df_model['early_default'].mean()



np.float64(0.022270720045975127)

In [10]:
df_model['early_default'].value_counts()

early_default
0    1932698
1      44023
Name: count, dtype: int64

In [11]:
df_model.shape

(1976721, 147)

# 10) Temporal Split (For Validation Planning)
# We define a time-based split to simulate real deployment:
# - Train: loans issued before 2015
# - Test: loans issued from 2015 onward
#
# NOTE: In this notebook we only *define* the split as a check. The actual modeling will be done in 03.


In [12]:
train = df_model[df_model['issue_d'] < '2015-01-01']
test  = df_model[df_model['issue_d'] >= '2015-01-01']

In [13]:
train.shape

(447707, 147)

In [14]:
test.shape

(1529014, 147)

# 11) Feature Whitelisting (Leakage Control)
# We **explicitly whitelist** features available at origination time.
# This is safer than "dropping leakage columns" because it prevents accidental inclusion of post-outcome fields.
#
# These features reflect:
# - loan attributes (amount, term, rate)
# - borrower attributes (income, DTI, employment)
# - credit history snapshot proxies (delinq_2yrs, inquiries, revolving utilization)
# - segmentation variables (purpose, home ownership, state)


In [15]:
origination_features = [
    'loan_amnt',
    'term',
    'int_rate',
    'installment',
    'grade',
    'sub_grade',
    'emp_length',
    'home_ownership',
    'annual_inc',
    'verification_status',
    'purpose',
    'dti',
    'delinq_2yrs',
    'inq_last_6mths',
    'open_acc',
    'pub_rec',
    'revol_bal',
    'revol_util',
    'total_acc',
    'addr_state'
]

# Keep only approved features + target + issue date (issue_d is used for temporal split later)
df_model_clean = df_model[origination_features + ['early_default', 'issue_d']]
df_model_clean = df_model_clean.copy()

In [16]:
df_model_clean.shape

(1976721, 22)

# 12) Missing Value Review
# We quickly review missingness to determine imputation strategy.

In [17]:
df_model_clean.isna().sum().sort_values(ascending=False).head(10)

emp_length        125091
revol_util          1495
dti                 1300
inq_last_6mths        30
pub_rec               29
total_acc             29
open_acc              29
delinq_2yrs           29
annual_inc             4
grade                  0
dtype: int64

# 13) Missing Value Treatment
# Strategy:
# - Numeric features → median imputation (robust to skew/outliers)
# - Categorical features → fill with "Unknown"
#
# We do NOT impute the target.

In [18]:
from sklearn.impute import SimpleImputer

numeric_cols = df_model_clean.select_dtypes(include=['int64','float64']).columns.tolist()
numeric_cols.remove('early_default')  # keep target untouched

categorical_cols = df_model_clean.select_dtypes(include=['object']).columns.tolist()

num_imputer = SimpleImputer(strategy='median')
df_model_clean.loc[:, numeric_cols] = num_imputer.fit_transform(df_model_clean[numeric_cols])

df_model_clean.loc[:, categorical_cols] = df_model_clean[categorical_cols].fillna('Unknown')


In [19]:
df_model_clean.isna().sum().sort_values(ascending=False).head(15)



loan_amnt              0
term                   0
int_rate               0
installment            0
grade                  0
sub_grade              0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
purpose                0
dti                    0
delinq_2yrs            0
inq_last_6mths         0
open_acc               0
dtype: int64

In [20]:
df_model_clean.dtypes

loan_amnt                       int64
term                           object
int_rate                      float64
installment                   float64
grade                          object
sub_grade                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
purpose                        object
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                       int64
revol_util                    float64
total_acc                     float64
addr_state                     object
early_default                   int64
issue_d                datetime64[ns]
dtype: object

# 14) Encode Categorical Variables
# We apply one-hot encoding for categorical variables. `drop_first=True` reduces redundancy and helps
# with models like logistic regression.
#
# Output: `df_model_encoded` is the final modeling dataset used by Notebook 03.



In [21]:
df_model_encoded = pd.get_dummies(df_model_clean, columns=categorical_cols, drop_first=True)
print(df_model_encoded.shape)


(1976721, 136)


# 15) Save Modeling Dataset
# Saving the processed dataset ensures:
# - reproducibility
# - consistent inputs across team members
# - clean separation between processing (02) and modeling (03)


In [22]:
# Save processed dataset for modeling
output_path = "../data/processed/early_default_modeling_dataset.csv"
df_model_encoded.to_csv(output_path, index=False)
print("Saved:", output_path)

Saved: ../data/processed/early_default_modeling_dataset.csv


In [23]:
pd.Series(origination_features).to_csv("../data/processed/origination_features_used.csv", index=False)
 